In [ ]:
from bids import BIDSLayout
import json
import pandas as pd
import os

from nilearn import image as nimg
from nilearn import plotting as nplot

import gzip
import matplotlib.pyplot as plt
import numpy as np

from scipy.signal import welch, detrend
from scipy.fft import rfft, rfftfreq



In [ ]:
 from Carl_load_responses import *

In [ ]:
responses, key = plot_all_responses_for_one_subject('sub-100', 'sp',downsampled=False,fs=14.40)

It can be seen that there are artifacts in these signals due to the pain ratings being adjusted for the first few volumes. As a result it would be good to discard the first four volumes (aka responses.)

In [ ]:
# Number of samples in normalized_tone

# Note the extra 'r' at the front
def plot_fft_participant(participant):
    responses, key = get_all_responses_for_one_subject(participant, 'sp',downsampled=False)
    demeaned = detrend(responses[:,4*36:],type='constant') #7, 8640
    plt.plot(demeaned.T)
    plt.show()
    yf = rfft(demeaned)
    xf = rfftfreq(demeaned.shape[1], 1 / 14.40)[:,np.newaxis]
    plt.plot(xf, np.mean(np.abs(yf).T,axis=1))
    plt.xlim([0,1])
    plt.show()
    
plot_fft_participant('sub-103')


## Now I estimate and plot the PSD for one subject across runs

There were 244 volumes taken, each of which took 2.5 seconds, and here we have 244*36 = 8784 recordings, so the sampling frequency was 1/(2.5/36) = 14.40 Hz

In [ ]:
def plot_welch_psd(subject_id,responses, key, fs, plot=True, xlim = [0,1]):

    f, Pxx_den = welch(responses,fs=fs, scaling = 'density', nperseg=256)
    if plot:    
        fig, ax = plt.subplots(figsize = (16,6))
        plt.semilogy(f, Pxx_den.T)
        plt.legend(key)
        plt.title(f"Welch's estimate of the PSD of subject {subject_id} responses across all sessions and runs.")
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Power')
        plt.xlim(xlim)
    return f, Pxx_den


## Using the non-downsampled responses to plot more accurate psds for the chronic patients

To investigate properly the validity of looking for information in the reponses of the participants, we can use the non downsampled responses of run 1 every visit that the participant attended, and form an average psd for that participant, then compare the psd's across all participants with chronic pain.

The highest frequency we can measure here comes frmo the nyquist criterion, and is fs/2 = 7.27 Hz 

In [ ]:
participants_df = pd.read_csv('openpain.org/subacute_longitudinal_study/participants.tsv', sep='\t')
chronic_df = participants_df.loc[participants_df["group"] == "chronic"]


In [ ]:
def plot_participant_psd(participant):
    font = 20
    responses, key = get_all_responses_for_one_subject(participant,'sp',downsampled = False)
    responses = responses.astype('float')
    #This method uses demeaned responses for the spectrum
    f, Pxx_den = welch(responses[:,4*36:], fs = 14.40, scaling='density')
    #now we average those psds to get a consistent view of one subject.
    fig, ax = plt.subplots(figsize = (16,6))
    plt.semilogy(f, Pxx_den.T)
    plt.legend(key,fontsize=font-5)
    plt.title(f"Welch's estimate of the PSD of subject {participant} responses across all sessions and runs.",fontsize=font)
    plt.xlabel('Frequency (Hz)',fontsize=font)
    plt.ylabel('Power',fontsize=font)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    #plt.xlim([0])
plot_participant_psd('sub-120')
plot_participant_psd('sub-100')
plot_participant_psd('sub-105')

Now we move to the computation of the average psd of all chronic subjects

In [ ]:
responses, key = get_all_responses_for_one_subject('sub-098','sp',downsampled = False, verbose = True)
# we have to remember to remove any spurious results from the responses - the length should be 8784 but
# there are also others in there for example the 8783 in sub-098. This will be removed here.
print([len(responses[i]) for i in range(len(responses))])
responses = np.array(list(filter(lambda x: len(x) == 8784, responses)))
print([len(responses[i]) for i in range(len(responses))])

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
Pxx_den_list = []
for subject in chronic_df['participant_id']:
    responses, key = get_all_responses_for_one_subject(subject,'sp',downsampled = False, verbose = False)
    responses = np.array(list(filter(lambda x: len(x) == 8784, responses))).astype('float')
    if len(responses.shape) != 2:
        print(subject,responses.shape)
        continue   
    f, Pxx_den = plot_welch_psd(subject,responses[:,4*36:],key, fs = 14.40, plot=False)

    #now we average those psds to get a single view of one subject.
    Pxx_den_list.append(np.mean(Pxx_den, axis=0))
    plt.semilogy(f, np.mean(Pxx_den, axis=0).T)
font = 25
plt.title(f"Average PSD for each chronic patient \nacross all sessions and runs.",fontsize=font)
plt.xlabel('Frequency (Hz)',fontsize=font)
plt.xticks(np.round(f[::],1),fontsize=font)
plt.yticks(fontsize=font)
plt.ylabel('Power/Hz',fontsize=font)
plt.xlim([0,1])
plt.show()

In [ ]:
from scipy.signal import savgol_filter

fig, ax = plt.subplots(figsize = (10,10))
Pxx_den_list = []
for subject in chronic_df['participant_id']:
    responses, key = get_all_responses_for_one_subject(subject,'sp',downsampled = False, verbose = False)
    responses = np.array(list(filter(lambda x: len(x) == 8784, responses))).astype('float')
    if len(responses.shape) != 2:
        print(subject,responses.shape)
        continue   
    f, Pxx_den = plot_welch_psd(subject,savgol_filter(responses[:,4*36:],4*36+1,3),key, fs = 14.40, plot=False)

    #now we average those psds to get a single view of one subject.
    Pxx_den_list.append(np.mean(Pxx_den, axis=0))
    plt.semilogy(f, np.mean(Pxx_den, axis=0).T)
plt.title(f"Average PSD for each chronic patient across all sessions and runs.")
plt.xlabel('Frequency (Hz)')
plt.xticks(f[::])

plt.ylabel('Power/Hz')
plt.xlim([0,1])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (16,6))
plt.stem(f, np.mean(np.array(Pxx_den_list), axis=0).T)
plt.title(f"PSD average of all chronic patient responses across all sessions and runs.")
plt.xlabel('Frequency (Hz)')
plt.xticks(f[::2])
plt.ylabel('Power/Hz')
plt.xlim([0,1])
plt.show()

## Examining the spectra of healthy controls

In [ ]:
subacute_df = participants_df.loc[participants_df["group"] == "subacute"]

fig, ax = plt.subplots(figsize = (16,6))
Pxx_den_list = []
for subject in subacute_df['participant_id']:
    responses, key = get_all_responses_for_one_subject(subject,'sp',downsampled = False, verbose = False)
    responses = np.array(list(filter(lambda x: len(x) == 8784, responses)))
    if len(responses.shape) != 2:
        print(subject,responses.shape)
        continue   
    f, Pxx_den = plot_welch_psd(subject,responses[:,4*36:],key, fs = 14.40, plot=False)

    #now we average those psds to get a consistent view of one subject.
    Pxx_den_list.append(np.mean(Pxx_den, axis=0))
    plt.semilogy(f, np.mean(Pxx_den, axis=0).T)
plt.title(f"Average of Welch's estimate of the PSD of subacute patient responses across all sessions and runs where this data was clean.")
plt.xlabel('Frequency (Hz)')
plt.xticks(f[::])

plt.ylabel('Power/Hz')
plt.xlim([0,1])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (16,6))
plt.stem(f, np.mean(np.array(Pxx_den_list), axis=0).T)
plt.title(f"PSD average of all subacute patient responses across all sessions and runs.")
plt.xlabel('Frequency (Hz)')
plt.xticks(f[::])
plt.ylabel('Power/Hz')
plt.xlim([0,1])
plt.show()

These spectra are indicative of the fact that the patient responses may be acting similarly to random walks.

## Plotting the autocorellation of patient responses

In [ ]:
import pandas
burn = 4*36
response = load_single_subject_response(subject = "sub-101", visit= 1, run= 1, plot = 0, task_type = 'sp', downsample = False)[burn:]
autocor = np.zeros(len(response))
for i in range(len(response)):
    autocor[i]=pandas.Series.autocorr(pandas.Series(response),lag=i)

plt.plot(response)
plt.show()
plt.plot(autocor)